![Gambar Youtuber](Youtuber.jpg)
# Content Based Top 50 Youtuber Indonesia
#### Oleh : Topik Zulkarnain
Notebook ini bertujuan untuk membuat sistem rekomendasi channel youtube berdasrkan content based nya. Analisis yang digunakan adalah dengan mencari kesamaan channel youtube berdasarkan Jumlah subscriber, tipe channel, dan rata-rata video yang ditonton dalam waktu satu minggu. Sehingga dapat dilihat youtuber mana yang paling mirip dengan youtuber lain secara konten dan dari situ dapat diambil insight rekomendasi channel yotube

# Data
Data yang digunakan adalah data dari noxinfluencer.com, untuk mendapatkan datanya terlebih dahulu akan digunakan teknik webscrapping yaitu teknik mendapatkan data langsung dari website yang dituju pada kasus ini noxinfluencer.com. Data yang diambil hanyalah 50 channel youtube saja dengan update data pada tanggal 20 September 2019

### Web Scrapping menggunakan BeautifulSoup

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
url = 'https://www.noxinfluencer.com/youtube-channel-rank/top-250-id-all-youtuber-sorted-by-subs-weekly'

In [3]:
#Jika statusnya 200 maka udah OK
response = requests.get(url)
response

<Response [200]>

In [4]:
#Mengurai HTML
soup = BeautifulSoup(response.content, 'html.parser')

In [5]:
#Outputnya itu jadi list
info_youtuber = soup.find_all('tr', 'item')
print(info_youtuber)

[<tr class="item"><td class="text rank"><span class="num top3 gold "> </span><span class="change unchanged " data-rank-change=""><span class="change-mark">   -  </span><span class="change-num"> </span></span></td><td class="profile"><a class="star-avatar" href="/youtube/channel/UCaKLg1ELiX0zTJ6Je3c5esA"> <img alt="Atta Halilintar" class="avatar" src="https://res06.bignox.com/noxinfluencer/youtube/avatar/b93175bb0b18fa3cefdd3356d8dab8a3.png?x-oss-process=image/resize,m_lfit,h_64,w_64"/> <span class="name kol-name" title="Atta Halilintar"> Atta Halilintar</span></a> </td> <td class="text category" title="Entertainment"> <a class="category-text" href="https://www.noxinfluencer.com/youtube-channel-rank/top-250-id-entertainment-youtuber-sorted-by-subs-weekly"> Entertainment </a> </td><td class="text followerNum with-num"><span class="num"> 18.9M </span><br/><span class="change up"><span class="change-mark">↑</span><span class="change-num">  1.6% </span></span></td><td class="text avgView wi

In [6]:
nama_channel = []
kategori_channel = []
jumlah_subscriber = []
average_view = []
for p in info_youtuber:
    nama = p.find('span','name kol-name').get_text()
    kategori = p.find('a','category-text').get_text()
    subscriber = p.find('td', 'text followerNum with-num').get_text()
    view = p.find('td', 'text avgView with-num').get_text()
    nama_channel.append(nama)
    kategori_channel.append(kategori)
    jumlah_subscriber.append(subscriber)
    average_view.append(view)

In [7]:
produk_dict = {
    'nama':nama_channel,
    'kategori': kategori_channel,
    'jumlah subscriber' : jumlah_subscriber,
    'average view' : average_view
}

In [8]:
df = pd.DataFrame(produk_dict, columns=['nama','kategori', 'jumlah subscriber','average view'])

In [9]:
df.head()

,nama,kategori,jumlah subscriber,average view
0,Atta Halilintar,Entertainment,18.9M ↑ 1.6%,1.84M ↓ 11.3%
1,Ricis Official,Entertainment,16.8M ↑ 2.1%,1.34M ↑ 1.7%
2,GEN HALILINTAR,Entertainment,11.9M ↑ 1.5%,1.12M ↑ 3.1%
3,Calon Sarjana,Entertainment,11.76M ↑ 2.6%,364.13K ↓ 25.9%
4,Rans Entertainment,Entertainment,10.1M ↑ 3.2%,971.6K ↓ 21.2%


Data yang didapat masih belum bersih terlihat dari feature jumlah subscriber dan average view masih mengandung angka yang tidak diperlukan dan juga masih terdapat string pada angka. Terlebih dahulu harus dilakukan proses data cleaning.
### Data Cleaning

In [10]:
#Menghilangkan huruf ataupun symbol dari data jumlah subcriber
df['jumlah subscriber'] = df['jumlah subscriber'].str[:5].replace(r'[M↑]','',regex=True).astype(float)

In [11]:
#untuk cleaning feature average view jauh lebih sulit, dikarenakan terdapat dua satuan yaitu juta (M), dan juga ribu (K)
# Terlebih dahulu menghilangkan symbol dan memotong string dari akhir kalimat
df['average view'] = df['average view'].str[:-7].replace(r'[↓↑]','',regex=True)

In [12]:
#Menghilangkan spasi yang ada pada akhir string
df['average view'] = df['average view'].str.replace(' ','')

In [13]:
#Menyeleksi satuan untuk M dan K, untuk satuan M maka hanya akan direplace dan diconvert menjadi float
#namun untuk satuan K agar memiliki nilai yang serupa dengan M maka setelah direplace dan diconvert harus dibagi 1000 
avg_view = pd.DataFrame()
for i in range(len(df['average view'])):
    cek = df['average view'][i]
    if cek[-1] == 'K':
        teger = float(cek[:-1])
        df['average view'][i] = teger/ 1000
    else:
        teger = float(cek[:-1])
        df['average view'][i] = teger

C:\Users\topikz064115\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\topikz064115\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [14]:
df.head()

,nama,kategori,jumlah subscriber,average view
0,Atta Halilintar,Entertainment,18.9,1.84
1,Ricis Official,Entertainment,16.8,1.34
2,GEN HALILINTAR,Entertainment,11.9,1.12
3,Calon Sarjana,Entertainment,11.7,0.36413
4,Rans Entertainment,Entertainment,10.1,0.9716


Data sudah siap untuk dianalisis

# Content Based Menggunakan Matrix Similarity
Sekarang akan dibentuk matriks similarity untuk liat kemiripan dari channel youtube berdasarkan kategorinya, jumlah subscriber, dan average view. Sehingga nantinya dapat dijadikan bahan untuk rekomendasinya.

In [15]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
df.head()

,nama,kategori,jumlah subscriber,average view
0,Atta Halilintar,Entertainment,18.9,1.84
1,Ricis Official,Entertainment,16.8,1.34
2,GEN HALILINTAR,Entertainment,11.9,1.12
3,Calon Sarjana,Entertainment,11.7,0.36413
4,Rans Entertainment,Entertainment,10.1,0.9716


Mengubah data kategori channel menjadi numerik dengan menggunakan teknik label encoding

In [16]:
df_new = df
df_new.index = df['nama']
df_new = df_new.drop(columns=['nama'])

In [17]:
df_new.head()

,kategori,jumlah subscriber,average view
nama,,,
Atta Halilintar,Entertainment,18.9,1.84
Ricis Official,Entertainment,16.8,1.34
GEN HALILINTAR,Entertainment,11.9,1.12
Calon Sarjana,Entertainment,11.7,0.36413
Rans Entertainment,Entertainment,10.1,0.9716


In [18]:
df_new['average view'] = df_new['average view'].astype(float)

In [19]:
le = preprocessing.LabelEncoder()

In [20]:
le.fit(df_new['kategori'])

LabelEncoder()

In [21]:
df_new['kategori'] = le.transform(df_new['kategori'])

In [35]:
result = pd.DataFrame(cosine_similarity(df_new,df_new), columns = df['nama'], index = df['nama'])
result

nama,Atta Halilintar,Ricis Official,GEN HALILINTAR,Calon Sarjana,Rans Entertainment,TRANS7 OFFICIAL,YtCrash,Indosiar,SAAIHALILINTAR,The Shiny Peanut,...,JalanTikus,CNN Indonesia,Edho Zell,Tips Pintar,Dimas Gepenk,Naisa Alifia Yuriza (N.A.Y),FATEH HALILINTAR,Mainan Anak LetsPlay,Hanin Dhiya,Kerajinan 5-Menit
nama,,,,,,,,,,,,,,,,,,,,,
Atta Halilintar,1.000000,0.999763,0.998144,0.995804,0.995981,0.990566,0.992834,0.988803,0.977376,0.989656,...,0.469266,0.549988,0.918630,0.916869,0.590166,0.495932,0.897449,0.411495,0.573159,0.630505
Ricis Official,0.999763,1.000000,0.998764,0.997544,0.996946,0.993269,0.994620,0.991707,0.975912,0.991916,...,0.481395,0.561569,0.923658,0.922536,0.594811,0.507098,0.898520,0.397587,0.579084,0.641090
GEN HALILINTAR,0.998144,0.998764,1.000000,0.998082,0.999583,0.995173,0.998143,0.994446,0.982807,0.996343,...,0.522170,0.599830,0.940961,0.939459,0.633816,0.547789,0.918419,0.416211,0.618865,0.676601
Calon Sarjana,0.995804,0.997544,0.998082,1.000000,0.997648,0.998935,0.997988,0.998190,0.970579,0.996925,...,0.526411,0.604089,0.940364,0.941038,0.616895,0.549166,0.904253,0.360076,0.605203,0.679728
Rans Entertainment,0.995981,0.996946,0.999583,0.997648,1.000000,0.995749,0.999275,0.995519,0.984712,0.998093,...,0.546445,0.622533,0.950326,0.948825,0.654730,0.571692,0.927944,0.421094,0.640594,0.697471
TRANS7 OFFICIAL,0.990566,0.993269,0.995173,0.998935,0.995749,1.000000,0.997825,0.999852,0.965522,0.997875,...,0.557276,0.632932,0.950101,0.951833,0.633233,0.578150,0.907635,0.338179,0.624138,0.705676
YtCrash,0.992834,0.994620,0.998143,0.997988,0.999275,0.997825,1.000000,0.998016,0.980466,0.999697,...,0.569333,0.644055,0.957978,0.957494,0.664800,0.592916,0.929398,0.399010,0.652896,0.716865
Indosiar,0.988803,0.991707,0.994446,0.998190,0.995519,0.999852,0.998016,1.000000,0.966211,0.998433,...,0.571454,0.646137,0.955218,0.956947,0.645411,0.592102,0.912893,0.341025,0.636799,0.717757
SAAIHALILINTAR,0.977376,0.975912,0.982807,0.970579,0.984712,0.965522,0.980466,0.966211,1.000000,0.978405,...,0.565722,0.637411,0.948078,0.940762,0.722721,0.597483,0.962910,0.571497,0.701549,0.710611


Dari matriks similarity dapat dilihat kemiripan antar youtube channel dengan variasi korelasi kemiripan dari 10% hingga 99%. Hal ini dapat terlihat bahwa channel entertainment akan begitu mirip dengan channel entertainment lainnya asal memiliki kualitas konten yang sama. Adapun terdapat channel yang berkualitas nya sama akan tetapi type channelnya berbeda maka kemiripannya akan lemah. Maka dari matriks ini dapat dijadikan bahan untuk rekomendasi.